In [1]:
import lightgbm as lgb
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
pd.set_option('max_columns', 100)
pd.set_option('max_rows', 200)
%matplotlib inline

In [2]:
train4 = pd.read_csv('train_pre.csv')

In [38]:
# train4['gender'] = train4['gender'].astype(int)
# train4['age'] = train4['age'].astype(int)

In [3]:
train4_id = train4[['user_id']]
train4_y = train4[['current_service']]
train4_x = train4.drop(['user_id', 'current_service'], axis=1)
train4_feature = train4_x.columns

In [4]:
from sklearn.preprocessing import LabelEncoder
le4 = LabelEncoder()
train4_y_le = le4.fit_transform(train4_y['current_service'])

In [5]:
train4_tra_x = train4_x[1:int(4/5*len(train4_x))]
train4_val_x = train4_x[int(4/5*len(train4_x)):]

In [6]:
train4_tra_y = train4_y_le[1:int(4/5*len(train4_x))]
train4_val_y = train4_y_le[int(4/5*len(train4_x)):]

In [11]:
column_str = ['service_type', 'is_mix_service', 'contract_type', 'is_promise_low_consume', 'net_service', 'gender']
train_data4 = lgb.Dataset(train4_tra_x, label=train4_tra_y, feature_name=list(train4_tra_x.columns), categorical_feature=column_str)
validate_data4 = lgb.Dataset(train4_val_x, label=train4_val_y, feature_name=list(train4_tra_x.columns), categorical_feature=column_str)
train_all4 = lgb.Dataset(train4_x, train4_y_le, feature_name=list(train4_tra_x.columns), categorical_feature=column_str)

In [10]:
params4 = {'learning_rate':0.1, 
          'lambda_l1':0.18,
          'lambda_l2':0.23,
          'max_depth':8,
          'num_leaves':120, 
           'n_estimators':3000, 
          'objective':'multiclass',
          'num_class':11
          }

base8_120: 0.154928 处理异常出账金额：0.167 对出账金额开方：0.157 

增加fee_mean年龄0改为负平均值: 0.1574 增加fee_mean: 0.154882

In [12]:
cv_result = lgb.cv(params4, train_all4, nfold=5, verbose_eval=50, early_stopping_rounds=50)

[50]	cv_agg's multi_logloss: 0.250273 + 0.00171677
[100]	cv_agg's multi_logloss: 0.201076 + 0.00146814
[150]	cv_agg's multi_logloss: 0.185806 + 0.00153774
[200]	cv_agg's multi_logloss: 0.177491 + 0.00145414


KeyboardInterrupt: 

In [14]:
lgb4 = lgb.train(params4, train_all4, verbose_eval=50)

In [15]:
test4 = pd.read_csv('test_pre.csv')

In [16]:
test4_id = test4[['user_id']]
test4_x = test4.drop(['user_id'], axis=1)

In [17]:
pre_test4 = lgb4.predict(test4_x)

In [18]:
pre_test4 = [list(x).index(max(x)) for x in pre_test4]
pre_test4 = le4.inverse_transform(pre_test4)

In [19]:
predict4 = test4_id
predict4['predict_service'] = pre_test4

In [20]:
predict4.to_csv('lgb_all1011_8_150.csv', index=None)